In [1]:
import numpy as np  
import pandas as pd

In [2]:
df=pd.read_csv('ratings_Electronics.csv')

In [3]:
df.head()

,AKM1MP6P0OYPR,0132793040,5.0,1365811200
0,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
1,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
2,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
3,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
4,A1QGNMC6O1VW39,0511189877,5.0,1397433600


In [4]:
df.rename(columns={"5.0":"Rating","AKM1MP6P0OYPR":"user_id","0132793040":"product_id"},inplace=True)

In [5]:
df.drop(['1365811200'],axis=1,inplace=True)

In [6]:
df.head()

,user_id,product_id,Rating
0,A2CX7LUOHB2NDG,0321732944,5.0
1,A2NWSAGRHCP8N5,0439886341,1.0
2,A2WNBOD3WNDNKT,0439886341,3.0
3,A1GI0U4ZRJA8WN,0439886341,1.0
4,A1QGNMC6O1VW39,0511189877,5.0


In [7]:
#Popularity based Recommendation System
df.isnull().sum()

user_id       0
product_id    0
Rating        0
dtype: int64

In [8]:
df.describe()


,Rating
count,7.824481e+06
mean,4.012337e+00
std,1.380910e+00
min,1.000000e+00
25%,3.000000e+00
50%,5.000000e+00
75%,5.000000e+00
max,5.000000e+00


In [28]:
counts = df['user_id'].value_counts()
df_new=df[df['user_id'].isin(counts[counts>=200].index)]

In [29]:
df_new.shape

(13862, 3)

In [30]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df_new, test_size = 0.3, random_state=0)
train_data.head()

,user_id,product_id,Rating
4089444,A1EVV74UQYVKRY,B004G6002M,5.0
7403351,A12DQZKRKTNF5E,B00DJNCS6O,4.0
4795977,AVPNQUVZWMDSX,B005B3VO24,5.0
2075887,ARBKYIVNYWK3C,B001IAYUM0,5.0
3327478,A1VQHH85U7PX0,B003EYV224,4.0


In [31]:
train_data.shape

(9703, 3)

In [32]:
test_data.shape

(4159, 3)

In [33]:
#Count of user_id for each unique product as recommendation score 
train_data_grouped = train_data.groupby('product_id').agg({'user_id': 'count'}).reset_index()
train_data_grouped.rename(columns = {'user_id': 'score'},inplace=True)
train_data_grouped.head()

,product_id,score
0,1400532655,1
1,1400699169,1
2,9043413585,1
3,9800359788,1
4,9981719005,1


In [34]:
#Sort the products on recommendation score 
train_data_sort = train_data_grouped.sort_values(['score', 'product_id'], ascending = [0,1]) 
      
#Generate a recommendation rank based upon score 
train_data_sort['Rank'] = train_data_sort['score'].rank(ascending=0, method='first') 
          
#Get the top 5 recommendations 
popularity_recommendations = train_data_sort.head(5) 
popularity_recommendations 

,product_id,score,Rank
1676,B000JMJWV2,15,1.0
3912,B003ES5ZUU,12,2.0
4368,B004CLYEDC,11,3.0
5559,B00829THK0,11,4.0
6024,B00A1DJO12,11,5.0


In [37]:
# Use popularity based recommender model to make predictions
def recommend(userid):     
    user_recommendations = popularity_recommendations 
          
    #Add user_id column for which the recommendations are being generated 
    user_recommendations['user_id'] = userid 
      
    #Bring user_id column to the front 
    cols = user_recommendations.columns.tolist() 
    cols = cols[-1:] + cols[:-1] 
    user_recommendations = user_recommendations[cols] 
          
    return user_recommendations 

In [38]:
find_recom = [15,121,53]   # This list is user choice.
for i in find_recom:
    print("Here is the recommendation for the userId: %d\n" %(i))
    print(recommend(i))    
    print("\n") 

Here is the recommendation for the userId: 15

      user_id  product_id  score  Rank
1676       15  B000JMJWV2     15   1.0
3912       15  B003ES5ZUU     12   2.0
4368       15  B004CLYEDC     11   3.0
5559       15  B00829THK0     11   4.0
6024       15  B00A1DJO12     11   5.0


Here is the recommendation for the userId: 121

      user_id  product_id  score  Rank
1676      121  B000JMJWV2     15   1.0
3912      121  B003ES5ZUU     12   2.0
4368      121  B004CLYEDC     11   3.0
5559      121  B00829THK0     11   4.0
6024      121  B00A1DJO12     11   5.0


Here is the recommendation for the userId: 53

      user_id  product_id  score  Rank
1676       53  B000JMJWV2     15   1.0
3912       53  B003ES5ZUU     12   2.0
4368       53  B004CLYEDC     11   3.0
5559       53  B00829THK0     11   4.0
6024       53  B00A1DJO12     11   5.0




C:\Users\shubhmad\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [39]:
df_CF = pd.concat([train_data, test_data]).reset_index()
df_CF.head()

,index,user_id,product_id,Rating
0,4089444,A1EVV74UQYVKRY,B004G6002M,5.0
1,7403351,A12DQZKRKTNF5E,B00DJNCS6O,4.0
2,4795977,AVPNQUVZWMDSX,B005B3VO24,5.0
3,2075887,ARBKYIVNYWK3C,B001IAYUM0,5.0
4,3327478,A1VQHH85U7PX0,B003EYV224,4.0


In [40]:
df_CF.shape


(13862, 4)

In [45]:
pivot_df = df_CF.pivot(index = 'user_id', columns ='product_id', values = 'Rating').fillna(0)
pivot_df.shape
pivot_df.head()

product_id,1400532655,1400699169,9043413585,9800359788,9981719005,998376234X,B000001OMI,B000001ON0,B00000DM9M,B00000DM9W,...,B00L21HC7A,B00L2442H0,B00L26YDA4,B00L2P3TRS,B00L3YHF6O,B00L43HAY6,B00L8I6SFY,B00LA6T0LS,B00LGQ6HL8,B00LI4ZZO8
user_id,,,,,,,,,,,,,,,,,,,,,
A11I1I9QLMAM1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A11KZ906QD08C5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A12DQZKRKTNF5E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A17BUUBOU0598B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1CMD08Z49PGKQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
pivot_df['user_index'] = np.arange(0, pivot_df.shape[0], 1)
pivot_df.head()

product_id,1400532655,1400699169,9043413585,9800359788,9981719005,998376234X,B000001OMI,B000001ON0,B00000DM9M,B00000DM9W,...,B00L2442H0,B00L26YDA4,B00L2P3TRS,B00L3YHF6O,B00L43HAY6,B00L8I6SFY,B00LA6T0LS,B00LGQ6HL8,B00LI4ZZO8,user_index
user_id,,,,,,,,,,,,,,,,,,,,,
A11I1I9QLMAM1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
A11KZ906QD08C5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
A12DQZKRKTNF5E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
A17BUUBOU0598B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
A1CMD08Z49PGKQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


In [47]:
from scipy.sparse.linalg import svds
# Singular Value Decomposition
U, sigma, Vt = svds(pivot_df, k = 10)
# Construct diagonal array in SVD
sigma = np.diag(sigma)

In [48]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 

# Predicted ratings
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = pivot_df.columns)
preds_df.head()

product_id,1400532655,1400699169,9043413585,9800359788,9981719005,998376234X,B000001OMI,B000001ON0,B00000DM9M,B00000DM9W,...,B00L2442H0,B00L26YDA4,B00L2P3TRS,B00L3YHF6O,B00L43HAY6,B00L8I6SFY,B00LA6T0LS,B00LGQ6HL8,B00LI4ZZO8,user_index
0,-0.024413,-0.022426,0.014553,0.193233,0.160547,0.000306,0.041726,-0.001349,-0.027343,0.000420,...,0.287259,0.219889,0.058853,0.486558,-0.013523,0.057208,0.104620,0.215053,0.082782,1.946146
1,-0.010978,0.006821,0.010002,0.071191,0.001961,0.001944,0.063732,0.005307,0.007248,0.000155,...,-0.038023,-0.014857,-0.027292,0.090378,0.011658,-0.021077,-0.044154,-0.045564,-0.013244,2.438410
2,-0.012240,-0.088942,-0.003475,0.176560,0.109291,-0.005875,-0.014259,0.000402,-0.102440,0.031334,...,0.670345,0.648503,0.130852,0.724975,-0.076028,0.120626,0.227353,0.418991,0.237875,3.506490
3,-0.012945,0.005607,0.014859,0.069972,0.151737,0.002553,0.139674,0.008916,0.002498,0.010188,...,0.009482,0.081737,-0.045499,0.448865,0.008770,-0.036249,-0.074498,-0.087135,-0.018197,4.361435
4,-0.023095,0.012908,0.006331,0.074459,0.004314,0.000827,-0.106187,0.029483,0.006059,0.028294,...,0.305074,0.336574,0.041530,0.454018,0.009949,0.049916,0.081463,0.115540,0.107642,5.756156


In [49]:

def recommend_items(userID, pivot_df, preds_df, num_recommendations):
      
    user_idx = userID-1 # index starts at 0
    
    # Get and sort the user's ratings
    sorted_user_ratings = pivot_df.iloc[user_idx].sort_values(ascending=False)
    #sorted_user_ratings
    sorted_user_predictions = preds_df.iloc[user_idx].sort_values(ascending=False)
    #sorted_user_predictions

    temp = pd.concat([sorted_user_ratings, sorted_user_predictions], axis=1)
    temp.index.name = 'Recommended Items'
    temp.columns = ['user_ratings', 'user_predictions']
    
    temp = temp.loc[temp.user_ratings == 0]   
    temp = temp.sort_values('user_predictions', ascending=False)
    print('\nBelow are the recommended items for user(user_id = {}):\n'.format(userID))
    print(temp.head(num_recommendations))

In [56]:
userID = 12
num_recommendations = 5
recommend_items(userID, pivot_df, preds_df, num_recommendations)


Below are the recommended items for user(user_id = 12):

                   user_ratings  user_predictions
Recommended Items                                
B00FSA8VQ2                  0.0          1.065522
B00ISFNSBW                  0.0          1.060532
B00G4UQ6U8                  0.0          0.947482
B0082E9K7U                  0.0          0.921640
B007OY5V68                  0.0          0.881847


C:\Users\shubhmad\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [52]:
preds_df.head()

product_id,1400532655,1400699169,9043413585,9800359788,9981719005,998376234X,B000001OMI,B000001ON0,B00000DM9M,B00000DM9W,...,B00L2442H0,B00L26YDA4,B00L2P3TRS,B00L3YHF6O,B00L43HAY6,B00L8I6SFY,B00LA6T0LS,B00LGQ6HL8,B00LI4ZZO8,user_index
0,-0.024413,-0.022426,0.014553,0.193233,0.160547,0.000306,0.041726,-0.001349,-0.027343,0.000420,...,0.287259,0.219889,0.058853,0.486558,-0.013523,0.057208,0.104620,0.215053,0.082782,1.946146
1,-0.010978,0.006821,0.010002,0.071191,0.001961,0.001944,0.063732,0.005307,0.007248,0.000155,...,-0.038023,-0.014857,-0.027292,0.090378,0.011658,-0.021077,-0.044154,-0.045564,-0.013244,2.438410
2,-0.012240,-0.088942,-0.003475,0.176560,0.109291,-0.005875,-0.014259,0.000402,-0.102440,0.031334,...,0.670345,0.648503,0.130852,0.724975,-0.076028,0.120626,0.227353,0.418991,0.237875,3.506490
3,-0.012945,0.005607,0.014859,0.069972,0.151737,0.002553,0.139674,0.008916,0.002498,0.010188,...,0.009482,0.081737,-0.045499,0.448865,0.008770,-0.036249,-0.074498,-0.087135,-0.018197,4.361435
4,-0.023095,0.012908,0.006331,0.074459,0.004314,0.000827,-0.106187,0.029483,0.006059,0.028294,...,0.305074,0.336574,0.041530,0.454018,0.009949,0.049916,0.081463,0.115540,0.107642,5.756156


In [59]:
final_ratings_matrix = df_new.pivot(index = 'user_id', columns ='product_id', values = 'Rating').fillna(0)
rmse_df = pd.concat([final_ratings_matrix.mean(), preds_df.mean()], axis=1)
rmse_df.columns = ['Avg_actual_ratings', 'Avg_predicted_ratings']
print(rmse_df.shape)
rmse_df['item_index'] = np.arange(0, rmse_df.shape[0], 1)
rmse_df.head()

(9674, 2)


C:\Users\shubhmad\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,Avg_actual_ratings,Avg_predicted_ratings,item_index
1400532655,0.019231,0.018808,0
1400699169,0.096154,0.095490,1
9043413585,0.057692,0.022574,2
9800359788,0.096154,0.108064,3
9981719005,0.057692,0.070822,4


In [60]:
RMSE = round((((rmse_df.Avg_actual_ratings - rmse_df.Avg_predicted_ratings) ** 2).mean() ** 0.5), 5)
print('\nRMSE SVD Model = {} \n'.format(RMSE))


RMSE SVD Model = 0.041 



In [62]:
userID = 20
num_recommendations = 5
recommend_items(userID, pivot_df, preds_df, num_recommendations)


Below are the recommended items for user(user_id = 20):

                   user_ratings  user_predictions
Recommended Items                                
B00DTZYHX4                  0.0          2.347371
B0082E9K7U                  0.0          2.270985
B00G4V0QSK                  0.0          2.011062
B0079UAT0A                  0.0          1.900312
B00CZDT30S                  0.0          1.834833


C:\Users\shubhmad\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [ ]:
Model-based Collaborative Filtering is a personalised recommender system, the recommendations are based on the past behavior of the user and it is not dependent on any additional information.
The Popularity-based recommender system is non-personalised and the recommendations are based on frequecy counts, which may be not suitable to the user.You can see the differance above for the user id 121 & 200, The Popularity based model has recommended the same set of 5 products to both but Collaborative Filtering based model has recommended entire different list based on the user past purchase history